In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np

# Load the combined narrative data
data_dir = '/content/drive/MyDrive/Colab Notebooks/Berkeley/266/FinalProject/Data/'
original_data = os.path.join(data_dir, 'sentences_with_labels.csv')

df = pd.read_csv(original_data)

# Fix Nan to ""
df = df.fillna("")

display(df.head())

print(df.shape)

,sentence1,sentence2,sentence3,sentence4,sentence5,mistral_sentence5,llama_sentence5,label
0,Frank had been drinking beer.,"He got a call from his girlfriend, asking wher...",Frank suddenly realized he had a date that night.,"Since Frank was already a bit drunk, he could ...",Frank spent the rest of the night drinking mor...,Frank called an Uber and was able to make it t...,"He called a taxi to pick him up, but it took a...",0
1,Dave was in the Bahamas on vacation.,He decided to go snorkeling on his second day.,"While snorkeling, he saw a cave up ahead.","He went into the cave, and he was terrified wh...","Dave swam away as fast as he could, but the sh...","Dave quickly swam back to the surface, scared ...",He swam back to the surface and quickly paddle...,1
2,Sunny enjoyed going to the beach.,"As she stepped out of her car, she realized sh...",It was quite sunny and she forgot her sunglasses.,Sunny got back into her car and heading toward...,Sunny found some sunglasses and headed back to...,Sunny bought a new pair of sunglasses at the m...,She went to the mall to buy a new pair of sung...,0
3,Sally was happy when her widowed mom found a n...,She discovered her siblings didn't feel the same.,Sally flew to visit her mom and her mom's new ...,"Although her mom was obviously in love, he was...",Sally went home and wondered about her parents...,"Although Sally struggled at first, she eventua...","Her mom's new husband seemed kind, but Sally n...",1
4,Dan hit his golf ball and watched it go.,The ball bounced on the grass and into the san...,Dan pretended that his ball actually landed on...,His friends were not paying attention so they ...,Dan snuck a ball on the green and made his put...,Dan's friends eventually noticed that his ball...,"His friends were impressed by his fake shot, b...",0


(40000, 8)


In [3]:
!pip install transformers datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

In [5]:
# Load your dataset
df = df[['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5',
         'mistral_sentence5', 'llama_sentence5', 'label']]

# Combine sentences into single inputs
def prepare_inputs(row):
    """
    Combine the context sentences with each possible ending into a format suitable for BERT.
    """
    mistral_input = f"{row['sentence1']} {row['sentence2']} {row['sentence3']} {row['sentence4']} {row['mistral_sentence5']}"
    llama_input = f"{row['sentence1']} {row['sentence2']} {row['sentence3']} {row['sentence4']} {row['llama_sentence5']}"
    return mistral_input, llama_input

df['mistral_input'], df['llama_input'] = zip(*df.apply(prepare_inputs, axis=1))

# Split dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Tokenizer and preprocessing
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples['mistral_input'],
        examples['llama_input'],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors='pt'
    )

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['mistral_input', 'llama_input', 'label']])
val_dataset = Dataset.from_pandas(val_df[['mistral_input', 'llama_input', 'label']])

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(['mistral_input', 'llama_input'])
val_dataset = val_dataset.remove_columns(['mistral_input', 'llama_input'])

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [8]:
# Model
from transformers import BertModel
import torch.nn as nn

class BertDenseClassifier(nn.Module):
    def __init__(self, bert_model_name, hidden_dim, num_classes):
        super(BertDenseClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()  # Add a loss function here

    def forward(self, input_ids, attention_mask, labels=None):
        # Get BERT embeddings
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_outputs.pooler_output  # CLS token representation

        # Pass through dense layers
        x = self.dropout(pooled_output)
        x = self.fc1(x)
        x = self.relu(x)
        logits = self.fc2(x)

        # Compute loss if labels are provided
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

def get_model(hidden_dim=256, num_classes=2):
    return BertDenseClassifier(bert_model_name='bert-base-uncased', hidden_dim=hidden_dim, num_classes=num_classes)

model = get_model(hidden_dim=256, num_classes=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
from graphviz import Digraph

def create_model_diagram():
    dot = Digraph(format="png", engine="dot")
    dot.attr(rankdir="LR", size="8,5")

    # Nodes
    dot.node("Mistral", "Mistral Input", shape="box", style="filled", fillcolor="lightblue")
    dot.node("Llama", "Llama Input", shape="box", style="filled", fillcolor="lightgreen")
    dot.node("BERT", "BERT", shape="ellipse", style="filled", fillcolor="orange")
    dot.node("Dense", "Dense Layer", shape="ellipse", style="filled", fillcolor="yellow")
    dot.node("Output", "Output", shape="box", style="filled", fillcolor="pink")

    # Edges
    dot.edge("Mistral", "BERT")
    dot.edge("Llama", "BERT")
    dot.edge("BERT", "Dense")
    dot.edge("Dense", "Output")

    return dot

# Create and render the diagram
diagram = create_model_diagram()
diagram.render("model_flow_diagram", view=True)  # Saves and opens the diagram



'model_flow_diagram.png'

In [12]:
from graphviz import Digraph

def create_model_diagram_no_dense():
    dot = Digraph(format="png", engine="dot")
    dot.attr(rankdir="LR", size="8,5")

    # Nodes
    dot.node("Mistral", "Mistral Input", shape="box", style="filled", fillcolor="lightblue")
    dot.node("Llama", "Llama Input", shape="box", style="filled", fillcolor="lightgreen")
    dot.node("BERT", "BERT", shape="ellipse", style="filled", fillcolor="orange")
    dot.node("Output", "Output", shape="box", style="filled", fillcolor="pink")

    # Edges
    dot.edge("Mistral", "BERT")
    dot.edge("Llama", "BERT")
    dot.edge("BERT", "Output")

    return dot

# Create and render the diagram
diagram_no_dense = create_model_diagram_no_dense()
diagram_no_dense.render("model_flow_no_dense_diagram", view=True)  # Saves and opens the diagram


'model_flow_no_dense_diagram.png'

In [ ]:
def get_trainer(
    model,
    train_dataset,
    val_dataset,
    tokenizer,
    learning_rate=2e-5,
    batch_size=8,
    weight_decay=0.01,
    num_train_epochs=5
):
    training_args = TrainingArguments(
        output_dir='./results',
        eval_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        save_strategy='epoch',
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        push_to_hub=False,
        report_to=["none"],  # Disable W&B logging
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = torch.argmax(torch.tensor(logits), dim=-1)
        accuracy = (predictions == labels).float().mean().item()
        return {'accuracy': accuracy}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    return trainer


In [ ]:
results = {'HiddenDim': [], 'Loss': [], 'Accuracy': []}
hidden_dims = [64, 128, 256, 512, 1024]

for hidden_dim in hidden_dims:
    print(f"Testing: HiddenDim={hidden_dim}")

    # Initialize a new model for each configuration
    model = get_model(hidden_dim=hidden_dim, num_classes=2)

    # Use with Trainer
    trainer = get_trainer(
        model=model,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        tokenizer=tokenizer,
        learning_rate=1e-5,
        batch_size=8
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    # Log results
    print(f"Validation Results for HiddenDim={hidden_dim}: {eval_results}")
    results['HiddenDim'].append(hidden_dim)
    results['Loss'].append(eval_results.get('eval_loss', None))
    results['Accuracy'].append(eval_results.get('eval_accuracy', None))


Testing: HiddenDim=64


<ipython-input-9-a0785899fce4>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.711200,0.682490,0.518625
2,0.568400,0.634914,0.629625
3,0.557000,0.665362,0.618875
4,0.434700,0.760998,0.617875
5,0.247300,0.902812,0.616625


Validation Results for HiddenDim=64: {'eval_loss': 0.634913980960846, 'eval_accuracy': 0.6296250224113464, 'eval_runtime': 23.0189, 'eval_samples_per_second': 347.54, 'eval_steps_per_second': 43.442, 'epoch': 5.0}
Testing: HiddenDim=128


<ipython-input-9-a0785899fce4>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.674600,0.653625,0.601250
2,0.607600,0.644528,0.619875
3,0.536300,0.705918,0.621875
4,0.524800,0.808063,0.618000
5,0.261700,1.067943,0.619500


Validation Results for HiddenDim=512: {'eval_loss': 0.70591801404953, 'eval_accuracy': 0.621874988079071, 'eval_runtime': 23.1007, 'eval_samples_per_second': 346.31, 'eval_steps_per_second': 43.289, 'epoch': 5.0}
Testing: HiddenDim=1024


<ipython-input-9-a0785899fce4>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.685700,0.682641,0.561250
2,0.585600,0.654402,0.615375
3,0.493600,0.705761,0.614250
4,0.439500,0.831876,0.608125
5,0.309900,1.040055,0.606125


Validation Results for HiddenDim=1024: {'eval_loss': 0.6544016599655151, 'eval_accuracy': 0.6153749823570251, 'eval_runtime': 23.1133, 'eval_samples_per_second': 346.122, 'eval_steps_per_second': 43.265, 'epoch': 5.0}


In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(data_dir, 'NovelResults.csv'), index=False)
display(results_df)

,HiddenDim,Loss,Accuracy
0,64,0.634914,0.629625
1,128,0.650621,0.615625
2,256,0.662499,0.624875
3,512,0.705918,0.621875
4,1024,0.654402,0.615375


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
# Print the evaluation results explicitly
print(f"Validation Results: {eval_results}")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.695200,0.691686,0.518625
2,0.688400,0.684496,0.560125
3,0.671800,0.673659,0.577625
4,0.642900,0.674377,0.579500
5,0.628500,0.672286,0.591375


Validation Results: {'eval_loss': 0.6722861528396606, 'eval_accuracy': 0.5913749933242798, 'eval_runtime': 16.9058, 'eval_samples_per_second': 473.21, 'eval_steps_per_second': 3.727, 'epoch': 5.0}


AttributeError: 'BertDenseClassifier' object has no attribute 'save_pretrained'

In [ ]:
from sklearn.metrics import classification_report

predictions = trainer.predict(val_dataset)

logits = predictions.predictions
predicted_classes = np.argmax(logits, axis=1)
true_classes = predictions.label_ids

report = classification_report(true_classes, predicted_classes, target_names=['Mistral', 'Llama'])
print(report)

              precision    recall  f1-score   support

     Mistral       0.62      0.54      0.58      4152
       Llama       0.57      0.65      0.60      3848

    accuracy                           0.59      8000
   macro avg       0.59      0.59      0.59      8000
weighted avg       0.60      0.59      0.59      8000



In [ ]:
from google.colab import runtime
runtime.unassign()